# Face detection and recognition training pipeline

The following example illustrates how to fine-tune an InceptionResnetV1 model on your own dataset. This will mostly follow standard pytorch training patterns.

In [1]:
from pathlib import Path
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import numpy as np
import os

#### Define run parameters

The dataset should follow the VGGFace2/ImageNet-style directory layout. Modify `data_dir` to the location of the dataset on wish to finetune on.

In [2]:
root_dir = Path(Path(__file__).parent if "__file__" in globals() else "").absolute().parent
data_dir = root_dir / "tests" / "data" / "test_images"

batch_size = 32
epochs = 8
workers = 0 if os.name == "nt" else 8

#### Determine if a nvidia GPU is available

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Running on device: {device}")

Running on device: cuda:0


#### Define MTCNN module

See `help(MTCNN)` for more details.

In [4]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

#### Perfom MTCNN facial detection

Iterate through the DataLoader object and obtain cropped faces.

In [5]:
dataset = datasets.ImageFolder(data_dir, transform=transforms.Resize((512, 512)))
dataset.samples = [
    (p, p.replace(str(data_dir), str(data_dir.parent / (data_dir.name + "_aligned"))))
        for p, _ in dataset.samples
]

loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    collate_fn=training.collate_pil
)

for i, (input_file, output_file) in enumerate(loader):
    mtcnn(input_file, save_path=output_file)
    print(f"\rBatch {i+1} of {len(loader)}", end="")
    
# Remove mtcnn to reduce GPU memory usage
del mtcnn

Batch 1 of 1

#### Define Inception Resnet V1 module

See `help(InceptionResnetV1)` for more details.

In [6]:
resnet = InceptionResnetV1(
    classify=True,
    pretrained="vggface2",
    num_classes=len(dataset.class_to_idx)
).to(device)

#### Define optimizer, scheduler, dataset, and dataloader

In [7]:
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [5, 10])

trans = transforms.Compose([
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])
dataset = datasets.ImageFolder(data_dir.parent / (data_dir.name + "_aligned"), transform=trans)
img_inds = np.arange(len(dataset))
np.random.shuffle(img_inds)
train_inds = img_inds[:int(0.8 * len(img_inds))]
val_inds = img_inds[int(0.8 * len(img_inds)):]

train_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(train_inds)
)
val_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(val_inds)
)

#### Define loss and evaluation functions

In [8]:
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    "fps": training.BatchTimer(),
    "acc": training.accuracy
}

#### Train model

In [9]:
writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10

print("\n\nInitial")
print("-" * 10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
)

for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    print("-" * 10)

    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

    resnet.eval()
    training.pass_epoch(
        resnet, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

writer.close()



Initial
----------
Valid |     1/1    | loss:    1.1656 | fps:    8.5033 | acc:    0.5000   

Epoch 1/8
----------
Train |     1/1    | loss:    1.4932 | fps:   23.3235 | acc:    0.2500   
Valid |     1/1    | loss:    0.8389 | fps:   13.2477 | acc:    0.5000   

Epoch 2/8
----------
Train |     1/1    | loss:    0.1318 | fps:   44.1008 | acc:    1.0000   
Valid |     1/1    | loss:    0.8094 | fps:   11.5341 | acc:    0.5000   

Epoch 3/8
----------
Train |     1/1    | loss:    0.0131 | fps:   47.5829 | acc:    1.0000   
Valid |     1/1    | loss:    1.0491 | fps:   11.4941 | acc:    0.5000   

Epoch 4/8
----------
Train |     1/1    | loss:    0.0045 | fps:   43.7946 | acc:    1.0000   
Valid |     1/1    | loss:    1.5456 | fps:   10.8716 | acc:    0.5000   

Epoch 5/8
----------
Train |     1/1    | loss:    0.0023 | fps:   43.7726 | acc:    1.0000   
Valid |     1/1    | loss:    2.1608 | fps:   10.1123 | acc:    0.5000   

Epoch 6/8
----------
Train |     1/1    | loss:    0.0